In [12]:
import os
from sqlalchemy import create_engine
import pandas as pd
from dotenv import load_dotenv
from icecream import ic
import json
load_dotenv()


True

## load variables
- data folder
- cleaned_df
- json template
- 

In [17]:
data_folder = os.path.join(os.getcwd(),'data')
cleaned_csv = os.path.join(data_folder, 'final_range_data.csv')
cleaned_df = pd.read_csv(cleaned_csv)

TEMPLATE_FILE = os.path.join(data_folder,"halfmoon_json_template.json")

with open(TEMPLATE_FILE, "r") as f:
    template = json.load(f)


### get names from db
the cell has two queries:
- get all somalia users
- get user details for json population

In [13]:
# create a db engine
engine = create_engine(os.getenv('DATABASE_URL1'))

#query 
query = '''select id, username, first_name, last_name, email, country from respi_regreeningusers where country in ('Somalia', 'somalia') order by first_name asc;'''
query2 = '''select id, first_name, last_name, gender, age_category, phone_number, email, country, organization, username from respi_regreeningusers where id in(3228,1831,3726,4247,4548,2495);'''
#execute the query and fetch the results
conn = engine.connect()
df = pd.read_sql_query(query, conn)
df2 = pd.read_sql_query(query2, conn)

#close the connection
conn.close()
engine.dispose()

#df

## enumerator
get user details from db for json population

In [14]:
for idx, i in enumerate(df2.to_dict(orient='records'), start=1):
    i['type'] = 'ENUMERATOR'
    i['id'] = idx
    ic(i)

ic| i: {'age_category': '18 – 35',
        'country': 'Somalia',
        'email': 'mrage@so-actionagainsthunger.org',
        'first_name': 'Muktar Mohamednoor',
        'gender': 'Male',
        'id': 1,
        'last_name': 'Rage',
        'organization': 'ACF',
        'phone_number': '252616051599',
        'type': 'ENUMERATOR',
        'username': 'mrage'}
ic| i: {'age_category': '',
        'country': 'Somalia',
        'email': 'khadhiib10@gmail.com',
        'first_name': 'Hassan Omar',
        'gender': 'Male',
        'id': 2,
        'last_name': 'Khadhib',
        'organization': 'Norwegian Refugee Council',
        'phone_number': '0615807566',
        'type': 'ENUMERATOR',
        'username': 'Khadhib'}
ic| i: {'age_category': '18 – 35',
        'country': 'Somalia',
        'email': 'suldaanyare1265@gmail.com',
        'first_name': 'Jamaal',
        'gender': 'Male',
        'id': 3,
        'last_name': 'Tagal',
        'organization': 'ACF',
        'phone_number': '2

## farmingEntity

In [18]:

#read and set values for farming entity
farming_entity = template[0]['farmingEntity'] 
ic(farming_entity)

for idx, i in enumerate(cleaned_df.to_dict(orient='records'), start=1):
    #ic(i['basic_info-organisation_name'])
    farming_entity['id'] = idx
    farming_entity['first_name'] = i['basic_info-organisation_name']
    farming_entity['middle_name'] = i['basic_info-organisation_name']
    farming_entity['last_name'] = i['basic_info-organisation_name']
    farming_entity['organization'] = 'INSTITUTION'
    ic(farming_entity)

ic| farming_entity: {'first_name': 'Test',
                     'id': 1,
                     'last_name': 'Test',
                     'middle_name': 'Test',
                     'organization': 'GROUP'}
ic| farming_entity: {'first_name': 'acf',
                     'id': 1,
                     'last_name': 'acf',
                     'middle_name': 'acf',
                     'organization': 'INSTITUTION'}
ic| farming_entity: {'first_name': 'concern',
                     'id': 2,
                     'last_name': 'concern',
                     'middle_name': 'concern',
                     'organization': 'INSTITUTION'}
ic| farming_entity: {'first_name': 'concern',
                     'id': 3,
                     'last_name': 'concern',
                     'middle_name': 'concern',
                     'organization': 'INSTITUTION'}
ic| farming_entity: {'first_name': 'concern',
                     'id': 4,
                     'last_name': 'concern',
                     'midd